# Тест на нормальное распределение

In [2]:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from itertools import combinations

In [3]:
fig_width = 6.3  
fig_height = 3.54  
plt.rcParams['figure.figsize'] = (fig_width, fig_height) # установка размеров для всех графиков

Фильтрация данных


In [4]:
file_path = "D:/AI/Computerized-campimetry-test-analysis/summary_per_color_group.xlsx"
data = pd.read_excel(file_path)
necessary = data[data["id_test_attempt"]==0]
test_data = necessary.rename(columns=lambda x: x.replace('T258', 'CM')) #замена названий столбцов

In [5]:
data.head()

,Unnamed: 0,T258_dH+_red_mean,T258_dH+_red_std,T258_dH+_red_min,T258_dH+_red_25%,T258_dH+_red_50%,T258_dH+_red_75%,T258_dH+_red_max,T258_dH+_yellow_green_mean,T258_dH+_yellow_green_std,...,T278_SMR_diff_25%,T278_SMR_diff_50%,T278_SMR_diff_75%,T278_SMR_diff_max,T278_SMR_corr,T278_SMR_tau,id_test_attempt,id_sample,age,sex
0,61005,2.75,1.500000,1,1.75,3.0,4.00,4,1.00,0.816497,...,-0.111650,-0.003722,0.110599,0.298893,-0.348606,-0.231483,0,0,20.010959,1
1,47000,2.25,0.957427,1,1.75,2.5,3.00,3,0.75,0.500000,...,-0.051903,0.048618,0.063985,0.219723,0.102884,0.066421,0,1,20.013699,0
2,47466,1.75,0.957427,1,1.00,1.5,2.25,3,0.50,0.577350,...,-0.064603,-0.051664,0.073810,0.274372,-0.202019,-0.016667,0,2,20.112329,0
3,59980,2.00,0.816497,1,1.75,2.0,2.25,3,0.75,0.500000,...,-0.055315,-0.027076,0.070588,0.181467,0.019172,-0.049460,0,3,20.158904,0
4,47927,2.00,0.816497,1,1.75,2.0,2.25,3,0.25,0.500000,...,-0.129545,0.003198,0.072435,0.324830,-0.149060,-0.191063,1,4,20.158904,0


In [6]:
test_data = test_data[["CM_dH+_yellow_green_50%", "CM_dH+_blue_50%","CM_dH+_purple_50%","CM_dH+_red_50%"]]
test_data.to_csv("test_data.csv",index = False) #сохранение для другого ноутбука
test_data

,CM_dH+_yellow_green_50%,CM_dH+_blue_50%,CM_dH+_purple_50%,CM_dH+_red_50%
0,1.0,4457.5,4610.0,3.0
1,1.0,4519.0,9925.5,2.5
2,0.5,9886.5,6593.0,1.5
3,1.0,3710.0,4141.0,2.0
6,0.0,8748.5,7961.5,3.0
...,...,...,...,...
2988,10.0,16521.0,2616.5,7.5
2989,5.5,11435.0,9460.5,7.5
2990,6.5,29358.5,16345.5,10.0
2991,3.5,15058.5,9581.5,13.0


## Построение графиков qq plot

Графики, которые помогают визуально оценить, насколько данные соответствуют нормальному распределению.Если данные соответствуют нормальному распределению, точки лягут вдоль прямой.

In [15]:
def build_qq_plots(data):
    for column in test_data.columns:
        plt.figure(figsize=(fig_width,fig_height), dpi=300)
        stats.probplot(data[column], dist="norm", plot=plt) 
        plt.title(f"Qq plot для {column}", fontsize=16, fontweight='light')
        plt.xlabel("Теоретические квантили", fontsize=14, fontweight='light')
        plt.ylabel("Упорядоченные значения", fontsize=14, fontweight='light')

## Тест Шапиро-Уилка

Статистический тест, который проверяет, насколько данные соответствуют нормальному распределению. Нулевая гипотеза (H0): данные имеют нормальное распределение. Альтернативная гипотеза (H1): данные не имеют нормального распределения.

In [18]:
def shapiro_wilk_test(data):
    columns = ["Столбец", "p-value", "Результат"]
    result_table = pd.DataFrame(columns = columns)
    for i, column in enumerate(data.columns):
        _, p_val = stats.shapiro(data[column]) # пропускается stat, где хранятся числа от 0 до 1(0- близко к нормальному, 1- сильное отклонение)
        if p_val < 0.05:
            status = "Гипотеза отклонена. Распределение не нормальное"
        else:
            status = "Гипотеза подтверждена. Распределение нормальное"
        result_table.loc[i] = [column, p_val, status]
    # Настройка ширины таблицы и вывод на экран
    pd.set_option('display.width', 130)
    print(result_table)            
    

In [19]:
shapiro_wilk_test(test_data)

                   Столбец       p-value                                        Результат
0  CM_dH+_yellow_green_50%  0.000000e+00  Гипотеза отклонена. Распределение не нормальное
1          CM_dH+_blue_50%  0.000000e+00  Гипотеза отклонена. Распределение не нормальное
2        CM_dH+_purple_50%  7.526754e-29  Гипотеза отклонена. Распределение не нормальное
3           CM_dH+_red_50%  0.000000e+00  Гипотеза отклонена. Распределение не нормальное


## Тест Краскала Уоллиса

Тест Краскала-Уоллиса — тест для проверки статистических различий между тремя и более независимыми группами. Он является аналогом ANOVA для данных, которые не следуют нормальному распределению. Нулевая гипотеза (H0): все группы имеют одинаковое распределение. Альтернативная гипотеза (H1): хотя бы одна из групп имеет другое распределение.

In [8]:
def kruskal_wallis_test_for_4_groups(data):
    groups = list()
    for col in data.columns:
        groups.append(data[col])

    _, p_val = stats.kruskal(*groups)

    print(f"p-value = {p_val}")

    if p_val < 0.05:
        print("Гипотеза отклонена. Есть статистические различия между группами")
    else:
        print("Гипотеза подтверждена. Все группы имеют одинаковое распределение")

In [9]:
kruskal_wallis_test_for_4_groups(test_data)

p-value = 0.0
Гипотеза отклонена. Есть статистические различия между группами


### Пост-хок анализ


Пост-хок анализ включает в себя попарные тесты Манна-Уитни с коррекцией Бонферрони на множественные сравнения, чтобы уменьшить вероятность ошибки первого рода. Тест Манна-Уитни — тест, который используется для сравнения двух независимых выборок, когда предположение о нормальности данных нарушается. Он проверяет, различаются ли медианы двух групп, или, более точно, можно ли сказать, что одна группа имеет систематически большие значения, чем другая. Коррекция Бонферрони — это метод, используемый для уменьшения вероятности ошибки первого рода (то есть, ложного положительного результата) при множественных тестах. 
Когда проводятся несколько статистических тестов одновременно, вероятность того, что хотя бы один из них покажет ложноположительный результат, увеличивается. Чтобы компенсировать увеличение вероятности ложных срабатываний при множественных тестах, уменьшается пороговое значение (alpha) для каждого теста, деля его на количество проведенных тестов. На практике в силу того, что в статистических пакетах мы работаем с p-value, корректируется именно его значение (умножается p-value каждого теста на количество тестов). Таким образом, мы просто сравниваем уже скорретированное p-value.

In [25]:
def post_hoc_analysis(data):
    col = data.columns #названия
    pairs = list(combinations(col, 2)) #генерирует все возможные пары из списка столбцов. список кортежей получится

    post_hoc_tab = []

    for pair in pairs:
        a = data[pair[0]]
        b = data[pair[1]]
        _,p_val = stats.mannwhitneyu(a,b)   #тест Манна Уитни для 2 групп

        post_hoc_tab.append({        #список словарей
                'Группа 1': pair[0],
                'Группа 2': pair[1],
                'p-value': p_val
            })

    post_hoc_df = pd.DataFrame(post_hoc_tab)
    post_hoc_df["p-value после коррекции"] = post_hoc_df["p-value"] * len(pairs) #коррекция Бонферрони (умножение на количество экспериментов)

    post_hoc_df['Результат'] = post_hoc_df['p-value после коррекции'].apply(
        lambda x: "Гипотеза отклонена. Различия есть" if x < 0.05 else "Гипотеза подтверждена. Различий нет")

    print(post_hoc_df)

In [26]:
post_hoc_analysis(test_data)

                  Группа 1           Группа 2        p-value  p-value после коррекции                          Результат
0  CM_dH+_yellow_green_50%    CM_dH+_blue_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть
1  CM_dH+_yellow_green_50%  CM_dH+_purple_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть
2  CM_dH+_yellow_green_50%     CM_dH+_red_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть
3          CM_dH+_blue_50%  CM_dH+_purple_50%  3.100112e-109            1.860067e-108  Гипотеза отклонена. Различия есть
4          CM_dH+_blue_50%     CM_dH+_red_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть
5        CM_dH+_purple_50%     CM_dH+_red_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть


Функция для реализации тестов

In [35]:
def stat_test (test, data, stat_tests_dict):
    if test in stat_tests_dict:
        stat_tests_dict[test](data)
    else:
        raise ValueError(f"Тест не найден. Доступные тесты: {stat_test_dict.keys()}")
    
   

In [37]:
 stat_tests_dict = {
                "post_hoc_analysis" : post_hoc_analysis,
                "kruskal_wallis_test_for_4_groups" : kruskal_wallis_test_for_4_groups,
                "shapiro_wilk_test" : shapiro_wilk_test
                 }
stat_test("post_hoc_analysis", test_data, stat_tests_dict)
     

                  Группа 1           Группа 2        p-value  p-value после коррекции                          Результат
0  CM_dH+_yellow_green_50%    CM_dH+_blue_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть
1  CM_dH+_yellow_green_50%  CM_dH+_purple_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть
2  CM_dH+_yellow_green_50%     CM_dH+_red_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть
3          CM_dH+_blue_50%  CM_dH+_purple_50%  3.100112e-109            1.860067e-108  Гипотеза отклонена. Различия есть
4          CM_dH+_blue_50%     CM_dH+_red_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть
5        CM_dH+_purple_50%     CM_dH+_red_50%   0.000000e+00             0.000000e+00  Гипотеза отклонена. Различия есть
